In [1]:
import flywheel
import datetime

# Create client
fw = flywheel.Client()

# define containers & files
gear = fw.lookup('gears/xcpengine-fw')
project = fw.lookup('davis/presurgicalEpilepsy')
design_file = project.get_file('cbf_WT.dsn')

In [2]:
# create subject list
fin = 'controls.txt'
sid_list = [line.rstrip('\n').split()[0] for line in open(fin)]
sid_list

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15']

In [9]:
for sub in sid_list[0:3]:
    ses = fw.lookup('davis/presurgicalEpilepsy/%s/ses-01' % (sub))
    print('Trying %s' % (fw.get(ses.parents['subject']).label))
    
    # get ASL and M0 files
    asl_img = None
    m0_img = None
    for acq in ses.acquisitions():
        if acq.files[0].classification['Measurement'] == ['Perfusion']:
            asl_container=fw.get(acq.id)
            for f in asl_container.files:
                if '.nii.gz' in f.name:
                    asl_img=f
        if 'M0' in acq.label:
            m0_container=fw.get(acq.id)
            for f in m0_container.files:
                if '.nii.gz' in f.name:
                    m0_img=f
        else:
            pass
        
    for analysis in ses.analyses:
        if analysis.job.state!='failed' and analysis.gear_info.name=='xcpengine-fw' and analysis.job.config['config']['analysis_type']=='struc':
            print(analysis.label)
            struct_output = [f for f in analysis.files if 'struc' in f.name]
            print(struct_output[0].name)
    
    
    if asl_img is not None and m0_img is not None:
        print('Running analysis for %s' % (sub))
        # set inputs and config for gear
        inputs = {'antsct': struct_output[0], 'designfile': design_file, 'img': asl_img, 'm0': m0_img}
        config = {'analysis_type': 'cbf'}
        #analysis_id = gear.run(analysis_label='WT_xcp cbf', config=config, inputs=inputs, destination=ses)

Trying C1
WT_xcp struc
xcpEngineouput_struc.zip
Running analysis for C1
Trying C2
WT_xcp struc
xcpEngineouput_struc.zip
Running analysis for C2
Trying C3
WT_xcp struc
xcpEngineouput_struc.zip
Running analysis for C3
